In [ ]:
%matplotlib inline
import numpy as np
import cv2
import os
import pandas as pd
from tqdm import tqdm
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix,plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D , BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from matplotlib import pyplot as plt
from skimage.morphology import extrema
from skimage.morphology import watershed as skwater
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
train = []
val = []
test = []
train_lbl = []
val_lbl = []
test_lbl = []
def loaddata(path,img_list,label_list, label):
    all_images = os.listdir(path)
    for image in tqdm(all_images):
        img = cv2.imread(os.path.join(path, image))
        img_list.append(img)
        label_list.append(label)  

### DATA INME YOK


In [ ]:
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/INME_YOK/train'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/INME_YOK/val'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/INME_YOK/test'

loaddata(path1,train,train_lbl,0)
loaddata(path2,val,val_lbl,0)
loaddata(path3,test,test_lbl,0)

100%|██████████| 1000/1000 [11:56<00:00,  1.40it/s]


### DATA ISKEMI

In [ ]:
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/ISKEMI/train'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/ISKEMI/val'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/ISKEMI/test'

loaddata(path1,train,train_lbl,1)
loaddata(path2,val,val_lbl,1)
loaddata(path3,test,test_lbl,1)

100%|██████████| 500/500 [00:01<00:00, 289.41it/s]


### DATA KANAMA

In [ ]:
path1 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/KANAMA/train'
path2 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/KANAMA/val'
path3 = '/content/gdrive/My Drive/TeknofestETAP1/DATA_PRO/KANAMA/test'

loaddata(path1,train,train_lbl,1)
loaddata(path2,val,val_lbl,1)
loaddata(path3,test,test_lbl,1)

100%|██████████| 500/500 [03:08<00:00,  2.65it/s]


In [ ]:
import random

def zipici(img, lbl):
    arifk = list(zip(img, lbl))
    random.shuffle(arifk)
    img, lbl = zip(*arifk)

    return img, lbl

train, train_lbl = zipici(train, train_lbl)
val, val_lbl = zipici(val, val_lbl)
test, test_lbl = zipici(test, test_lbl)

In [ ]:
x_train = np.array(train)
y_train = np.array(train_lbl)
x_val = np.array(val)
y_val = np.array(val_lbl)
x_test = np.array(test)
y_test = np.array(test_lbl)

# ResNet50 MODEL

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
Resnet=ResNet50(include_top=False,pooling='max', weights="imagenet")

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
modelRes = Sequential()
modelRes.add(Resnet)
modelRes.add(Flatten())
modelRes.add(Dense(512, activation='relu'))
modelRes.add(Dropout(0.2))
modelRes.add(Dense(256, activation='relu'))
modelRes.add(Dropout(0.2))
modelRes.add(Dense(128, activation='relu'))
modelRes.add(Dropout(0.2))
modelRes.add(Dense(64, activation='relu'))
modelRes.add(Dropout(0.2))
modelRes.add(Dense(32, activation='relu'))
modelRes.add(Dropout(0.2))
modelRes.add(Dense(1, activation='sigmoid'))
modelRes.layers[0].trainable=False
# Compile model 
modelRes.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',
                                                                      tf.keras.metrics.Precision(),
                                                                      tf.keras.metrics.Recall()])

In [ ]:
history = modelRes.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=10, epochs=60)

Epoch 1/60
153/720 [=====>........................] - ETA: 22s - loss: 0.6932 - accuracy: 0.4935 - precision: 0.4987 - recall: 0.4942

KeyboardInterrupt: ignored

In [ ]:
scores = modelRes.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

11/11 [==============================] - 2s 151ms/step - loss: 0.5548 - accuracy: 0.9099 - precision: 0.9348 - recall: 0.7818
Test loss: 0.5548179745674133
Test accuracy: 0.9099099040031433


In [ ]:
#PATH DEĞİŞTİR HER EĞİTİMDEN SONRA !!!!!!!!!!!!!!!!!!!!
modelRes.save('/content/gdrive/My Drive/TeknofestETAP1/Weights/ResNet/resnet_yes_yes.h5') 

#  LOAD MODEL
#model1 = load_model('brain_tumor_vgg16.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
ynew = modelRes.predict_classes(X_test)
count = 0
index = 0
for img in X_test[100:150]:
    plt.figure()
    plt.title(ynew[index])
    plt.xlabel(y_test[index])
    plt.imshow(img)
    index = index +1